<a href="https://colab.research.google.com/github/nahumsa/DM-Reconstruction/blob/master/QMetrics%20Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
print(tf.__version__)

2.2.0


# Tests

In [0]:
#Creating pauli matrices
sigma_0_np = np.array([[1.,0.],
                      [0.,1.]], dtype=np.complex64)

sigma_1_np = np.array([[0.,1.],
                       [1.,0.]], dtype=np.complex64)

sigma_2_np = np.array([[0.,1.j],
                       [-1.j,0.]], dtype=np.complex64)

sigma_3_np = np.array([[1.,0.],
                       [0.,-1.]], dtype=np.complex64)

#Converting to tensors
sigma_0 = tf.Variable(sigma_0_np, tf.complex64)

sigma_1 = tf.Variable(sigma_1_np, tf.complex64)

sigma_2 = tf.Variable(sigma_2_np, tf.complex64)

sigma_3 = tf.Variable(sigma_3_np, dtype=tf.complex64)

In [0]:
def tf_kron(a: tf.Tensor,
            b: tf.Tensor) -> tf.Tensor:

    """Calculates the Kronocker product of two matrices ((2,2) Tensors).

    Parameters
    -----------------------------------------------------------------------
    a(tf.Tensor): Tensor on the left of the product.
    b(tf.Tensor): Tensor on the right of the product.

    Return
    -----------------------------------------------------------------------
    (tf.tensor): Kronocker product between a and b.

    """
    assert len(a.shape) == 2
    assert len(b.shape) == 2
    a_shape = list(b.shape)    
    b_shape = list(b.shape)
    return tf.reshape(tf.reshape(a,[a_shape[0],1,a_shape[1],1])*tf.reshape(b,[1,b_shape[0],1,b_shape[1]]),[a_shape[0]*b_shape[0],a_shape[1]*b_shape[1]])

In [0]:
from itertools import product
name_basis_1 = ['I', 'X', 'Y', 'Z']
basis_1 = [sigma_0, sigma_1,sigma_2,sigma_3]
name_basis_2 = []
basis_2 = []
for (name_1, meas_1),(name_2,meas_2) in product(zip(name_basis_1, basis_1),zip(name_basis_1, basis_1)):
  if name_1 == 'I' and name_2 == 'I':
    pass
  else:
    basis_2.append(tf_kron(meas_1,meas_2))
    name_basis_2.append(name_1 + name_2)

basis_2_tf = tf.Variable(basis_2)
print(name_basis_2)
print(basis_2_tf.shape)

['IX', 'IY', 'IZ', 'XI', 'XX', 'XY', 'XZ', 'YI', 'YX', 'YY', 'YZ', 'ZI', 'ZX', 'ZY', 'ZZ']
(15, 4, 4)


In [0]:
meas = [np.random.rand(15),np.random.rand(15),np.random.rand(15),np.random.rand(15)]
measurement = np.array(meas, dtype=np.complex64)
measurement_tensor = tf.Variable(measurement, tf.complex64)
#print(measurement_tensor)

In [0]:
ones_II = tf.ones((measurement_tensor.shape[0],1), dtype=tf.dtypes.complex64)
II = tf.Variable([tf_kron(sigma_0 , sigma_0)])
density_matrix = 0.25*(tf.tensordot(ones_II , II ,axes=1) + tf.tensordot(measurement_tensor,basis_2_tf,axes=1))
print(density_matrix.shape)

(4, 4, 4)


In [0]:
mat = tf.squeeze(density_matrix)
#print(mat)
print(f'Eigenvalues: {tf.linalg.eigvalsh(mat)}')
log_mat = tf.linalg.logm(mat)

Eigenvalues: [[-0.43327442+0.j -0.01793911+0.j  0.26848564+0.j  1.1827275 +0.j]
 [-0.5466988 +0.j -0.0033648 +0.j  0.30324015+0.j  1.2468239 +0.j]
 [-0.24974662+0.j -0.00860144+0.j  0.32327715+0.j  0.93507105+0.j]
 [-0.31569844+0.j -0.12145308+0.j  0.45409948+0.j  0.98305184+0.j]]


# Methods

## One qubit

In [0]:
def create_density_mat(measurements: np.array) -> tf.Tensor:
  
  #Creating the basis
  sigma_0_np = np.array([[1.,0.],
                        [0.,1.]], dtype=np.complex64)

  sigma_1_np = np.array([[0.,1.],
                         [1.,0.]], dtype=np.complex64)

  sigma_2_np = np.array([[0.,1.j],
                         [-1.j,0.]], dtype=np.complex64)

  sigma_3_np = np.array([[1.,0.],
                        [0.,-1.]], dtype=np.complex64)

  #Converting to tensors
  sigma_0 = tf.Variable(sigma_0_np, tf.complex64)

  sigma_1 = tf.Variable(sigma_1_np, tf.complex64)

  sigma_2 = tf.Variable(sigma_2_np, tf.complex64)

  sigma_3 = tf.Variable(sigma_3_np, dtype=tf.complex64)

  basis = tf.Variable([sigma_1,sigma_2,sigma_3])
  
  measurement = np.array(measurements, dtype=np.complex64)
  measurement_tensor = tf.Variable(measurement, tf.complex64)

  density_matrix = 0.5*(sigma_0 + tf.tensordot(measurement_tensor,basis,axes=1))
  return density_matrix

In [0]:
def trace_dist(A,B):
  dif = A - B
  dif = tf.transpose(dif, conjugate=True, perm=[0,2,1]) * dif  
  vals = tf.linalg.eigvalsh(dif)
  return tf.math.real(0.5*tf.reduce_sum(tf.math.sqrt(tf.math.abs(vals)),axis=-1))

In [0]:
def trace_loss(y_true,y_pred):
  d_y_true = create_density_mat(y_true)  
  d_y_pred = create_density_mat(y_pred)  
  return tf.reduce_mean(trace_dist(d_y_pred,d_y_true))

In [0]:
def entropy(A):    
  eigen_A = tf.linalg.eigvalsh(A)
  return -tf.math.real(tf.reduce_sum(eigen_A*tf.math.log(eigen_A),axis=-1))

def relative_entropy(A,B):
  log_B = tf.linalg.logm(B)
  eigen_AlogB = tf.linalg.eigvalsh(A*log_B)
  return - entropy(A) - tf.math.real(tf.reduce_sum(eigen_AlogB,axis=-1)) 

In [0]:
def r_entropy_loss(y_true,y_pred):
  d_y_true = create_density_mat(y_true)  
  d_y_pred = create_density_mat(y_pred)  
  return tf.reduce_mean(relative_entropy(d_y_pred,d_y_true))

In [0]:
y_1 = [[.5,.5,0],[.2,0.3,.3]]
y_2 = [[1,0,0],[.2,.3,.3]]
r_entropy_loss(y_true=y_1, y_pred=y_2)

<tf.Tensor: shape=(), dtype=float32, numpy=-0.12907417>

## Two qubits

In [0]:
from itertools import product
def tf_kron(a: tf.Tensor,
            b: tf.Tensor) -> tf.Tensor:

  """Calculates the Kronocker product of two matrices ((2,2) Tensors).

  Parameters
  -----------------------------------------------------------------------
  a(tf.Tensor): Tensor on the left of the product.
  b(tf.Tensor): Tensor on the right of the product.

  Return
  -----------------------------------------------------------------------
  (tf.tensor): Kronocker product between a and b.

  """
  assert len(a.shape) == 2
  assert len(b.shape) == 2
  a_shape = list(b.shape)    
  b_shape = list(b.shape)
  return tf.reshape(tf.reshape(a,[a_shape[0],1,a_shape[1],1])*tf.reshape(b,[1,b_shape[0],1,b_shape[1]]),[a_shape[0]*b_shape[0],a_shape[1]*b_shape[1]])


#Creating pauli matrices
sigma_0_np = np.array([[1.,0.],
                      [0.,1.]], dtype=np.complex64)

sigma_1_np = np.array([[0.,1.],
                       [1.,0.]], dtype=np.complex64)

sigma_2_np = np.array([[0.,1.j],
                       [-1.j,0.]], dtype=np.complex64)

sigma_3_np = np.array([[1.,0.],
                       [0.,-1.]], dtype=np.complex64)

#Converting to tensors
sigma_0 = tf.Variable(sigma_0_np, tf.complex64)

sigma_1 = tf.Variable(sigma_1_np, tf.complex64)

sigma_2 = tf.Variable(sigma_2_np, tf.complex64)

sigma_3 = tf.Variable(sigma_3_np, dtype=tf.complex64)

def create_2qubit_density_mat(measurements: tf.Variable) -> tf.Variable:
  
  name_basis_1 = ['I', 'X', 'Y', 'Z']
  basis_1 = [sigma_0, sigma_1,sigma_2,sigma_3]
  name_basis_2 = []
  basis_2 = []
  for (name_1, meas_1),(name_2,meas_2) in product(zip(name_basis_1, basis_1),zip(name_basis_1, basis_1)):
    if name_1 == 'I' and name_2 == 'I':
      pass
    else:
      basis_2.append(tf_kron(meas_1,meas_2))
      name_basis_2.append(name_1 + name_2)
  
  basis_2_tf = tf.Variable(basis_2)

  # Helper to make tr(density_matrix) = 1
  ones_II = tf.ones((tf.shape(measurements)[0],1), dtype=tf.dtypes.complex64)
  II = tf.Variable([tf_kron(sigma_0 , sigma_0)])
  
  density_matrix = 0.25*(tf.tensordot(ones_II , II ,axes=1) + tf.tensordot(measurements,basis_2_tf,axes=1))
  return density_matrix

In [0]:
def trace_dist(A,B):
  dif = tf.math.subtract(A,B)
  
  dif = tf.transpose(dif, conjugate=True, perm=[0,2,1]) * dif  
  vals = tf.linalg.eigvalsh(dif)
  return tf.math.real(0.5*tf.reduce_sum(tf.math.sqrt(tf.math.abs(vals)),axis=-1))

def trace_loss(y_true,y_pred):
  d_y_true = create_2qubit_density_mat(y_true)  
  d_y_pred = create_2qubit_density_mat(y_pred)    
  return tf.reduce_mean(trace_dist(d_y_pred,d_y_true))

In [0]:
def entropy(A):    
  eigen_A = tf.linalg.eigvalsh(A)
  return -tf.math.real(tf.reduce_sum(eigen_A*tf.math.log(eigen_A),axis=-1))

def relative_entropy(A,B):
  log_B = tf.linalg.logm(B)
  eigen_AlogB = tf.linalg.eigvalsh(tf.tensordot(A,log_B, axes=1))
  return - entropy(A) - tf.math.real(tf.reduce_sum(eigen_AlogB,axis=-1)) 

def r_entropy_loss(y_true,y_pred):
  d_y_true = create_2qubit_density_mat(y_true)  
  d_y_pred = create_2qubit_density_mat(y_pred)  
  return tf.reduce_mean(relative_entropy(d_y_pred,d_y_true))

In [6]:
meas_1 = [np.random.rand(15),np.random.rand(15),np.random.rand(15)]
measurement_1 = np.array(meas_1, dtype=np.complex64)
measurement_tensor_1 = tf.Variable(measurement_1, tf.complex64)

meas_2 = [np.random.rand(15),np.random.rand(15),np.random.rand(15)]
measurement_2 = np.array(meas_2, dtype=np.complex64)
measurement_tensor_2 = tf.Variable(measurement_2, tf.complex64)

print(entropy(create_2qubit_density_mat(measurement_tensor_1)))
print(trace_loss(measurement_tensor_1,measurement_tensor_2))
#print(r_entropy_loss(measurement_tensor_1,measurement_tensor_2))

tf.Tensor([-0.65438056 -0.12190735  0.0870981 ], shape=(3,), dtype=float32)
tf.Tensor(0.5531197, shape=(), dtype=float32)


## Fidelity

In [0]:
meas_1 = [np.random.rand(15),np.random.rand(15),np.random.rand(15)]
measurement_1 = np.array(meas_1, dtype=np.complex64)
measurement_tensor_1 = tf.Variable(measurement_1, tf.complex64)

meas_2 = [np.random.rand(15),np.random.rand(15),np.random.rand(15)]
measurement_2 = np.array(meas_2, dtype=np.complex64)
measurement_tensor_2 = tf.Variable(measurement_2, tf.complex64)

In [0]:
def fidelity(A,B):
  """ Calculates the fidelity between density matrices A and B.
  """
  sqrt_A = tf.linalg.sqrtm(A)  
  aux = sqrt_A@B@sqrt_A
  internal = tf.linalg.sqrtm(aux)
  eigenvalues = tf.linalg.eigvalsh(internal)  
  return tf.math.real(tf.reduce_sum(tf.sqrt(eigenvalues), axis=-1))

In [78]:
fidelity(create_2qubit_density_mat(measurement_tensor_1),create_2qubit_density_mat(measurement_tensor_1))

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([2.1140258, 1.7614529, 1.8742006], dtype=float32)>

In [0]:
def dagger(A):
  """Computes the transpose conjulgate of a density matrix A.
  
  """
  return tf.transpose(A, perm=[0,2,1], conjugate=True)    

def fidelity(A,B):    
  """ Calculates the fidelity between density matrices A and B.
  """
  (eigenvalues_A, vector_A) = tf.linalg.eig(A)   
  aux_fidelity = vector_A@tf.linalg.diag(tf.math.sqrt(eigenvalues_A))@dagger(vector_A)    
  f_b = aux_fidelity@B@dagger(aux_fidelity)
  (eigenvalues_F, vector_F) = tf.linalg.eig(f_b)    
  return tf.square(tf.math.real(tf.linalg.trace(vector_F@tf.linalg.diag(tf.math.sqrt(eigenvalues_F))@dagger(vector_F))))

In [86]:
fidelity(create_2qubit_density_mat(measurement_tensor_1),create_2qubit_density_mat(measurement_tensor_1))

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([2.4242904, 1.9064056, 2.0652583], dtype=float32)>

In [0]:
def sqrt_svd(A):
  S, left_A, right_A = tf.linalg.svd(density)
  S = tf.cast(tf.linalg.diag(tf.sqrt(S)),dtype='complex64')
  return left_A@S@dagger(right_A)

def fidelity(A,B):
  sqrt_A = sqrt_svd(A)
  internal = sqrt_A@B@sqrt_A
  sqrt_internal = sqrt_svd(internal)
  return tf.square(tf.math.real(tf.linalg.trace(sqrt_internal)))

In [89]:
fidelity(create_2qubit_density_mat(measurement_tensor_1),create_2qubit_density_mat(measurement_tensor_1))

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.37355238, 1.0448463 , 1.385976  ], dtype=float32)>